# Avito Demand Prediction Challenge
-> O que é<br/>
-> O que queremos fazer<br/>
-> Como pensamos em fazer, utilizando quais arquivos


In [1]:
#importing important modules
import numpy as np # linear algebra
import pandas as pd
from pandas import read_csv # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn import tree

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
# Any results you write to the current directory are saved as output.

* Explorar, visualizar,
* sklearn preprocess encode
* treino arvore


Lendo CSVs

In [2]:
#reading the csv's
train = read_csv('../input/train.csv')
test = read_csv('../input/test.csv')

print ('Data loaded')

In [3]:
print('Train Columns ',train.columns)
print('Test Columns ',test.columns)

In [4]:
#dropping some columns
train = train.drop(['image', 'image_top_1'], axis = 1)
test = test.drop(['image', 'image_top_1'], axis = 1)

Y_df = train['deal_probability']
train_df = train.drop(['item_id','activation_date','user_type', 'title','description','deal_probability'], axis = 1)
enc_df = train.append(test).drop(['item_id','activation_date','user_type', 'title','description','deal_probability'], axis = 1)

In [5]:
print ('Train shape: ',train.shape)
print ('Test shape: ',test.shape)

In [6]:
train[:10]

# O que será testado
Uma visualização melhor dos dados pode mostrar quais colunas são realmente importantes para o target. Logo de cara, *user_type* ou *user_type* não apresentam, intuitivamente, influência sobre a probabilidade. A Descrição e o título, por sua vez, tratam-se de textos e configuram um cenário que há de ser tratado distintamente dos demais dados categóricos.

Um bom teste seria realizar um treino base com as colunas base.

In [7]:
#transforming columns
enc = {}
for col in train_df.columns:
    enc[col] = LabelEncoder()
#     print (col)
#     training with all data to avoid "y unknown values" when encoding the test data
    enc[col].fit(enc_df[col].values.astype(str))
#     encoding the train data
    train_df[col] = enc[col].transform(train_df[col].values.astype(str))
y_enc = LabelEncoder()
Y_df = y_enc.fit_transform(Y_df)

print ("Label Encoding done")

In [8]:
train_size = 10000


In [9]:
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer, CountVectorizer
from nltk.corpus import stopwords

desc_df = train['description'][:50000]
print ('Starting vectorizing')
hash_vect = HashingVectorizer(n_features = 2**10, encoding = 'KOI8-R', stop_words = stopwords.words('russian'))
count_vect = CountVectorizer(encoding = 'KOI8-R', stop_words = stopwords.words('russian'))
# tfidf_transf = TfidfTransformer()

desc_vec = hash_vect.fit_transform(desc_df.values.astype(str))
sparse_desc = pd.SparseSeries(desc_vec.toarray().ravel(), fill_value=0)
print ("Vectorizing done")
# print (desc_df.values.astype(str))
# desc_vec = count_vect.fit_transform(desc_df.values.astype(str))

# desc_vec = tfidf_transf.fit_transform(cv)

# Iniciando o treinamento com o dataset
Nesta parte, será iniciado o treinamento com o dataset que foi pré-processado acima. Nós estamos utilizando *colocar os métodos utilizados aqui e seus motivos*

In [10]:
train_df_fit = train_df[:train_size]
# print (desc_vec.todense()[0].)
train_df_fit['description'] = sparse_desc[:train_size]
# train_df_fit['description'] = desc_vec[:train_size]

Y_df_fit = Y_df[:train_size]

print ('Train shape: ',train_df_fit.shape)
print ('Test shape: ',test.shape)
print ('Y_df: ', Y_df_fit.shape)

In [11]:
#fazendo o treinamento com fit
clf = tree.DecisionTreeClassifier()
print (train_df_fit.columns)
clf = clf.fit(train_df_fit, Y_df_fit)

In [12]:
#testando o quanto o modelo consegue acertar com score
train_df_test = train_df[10000:15000]
train_df_test['description'] = sparse_desc[10000:15000]
Y_df_test = Y_df[10000:15000]
clf.score(train_df_test, Y_df_test)

In [13]:
#tirando colunas que não foram utilizadas
test_size = 1000
# print('Test labels: ', test)
desc_test = test['description'][:test_size]
test_df = test.drop(['item_id','activation_date','user_type', 'title','description'], axis = 1)
test_df = test_df[:test_size]
print('New test: ', test_df.shape)


In [14]:
#fazendo encoder
for col in test_df.columns:
    print (col)
    test_df[col] = enc[col].transform(test_df[col].values.astype(str))

desc_test_vec = hash_vect.transform(desc_test.values.astype(str))
test_df['description'] = pd.SparseArray(desc_test_vec.toarray().ravel(), fill_value = 0)[:test_size]


In [15]:
#predict
res = clf.predict(test_df)
print(res)

In [16]:
y_enc.inverse_transform(res)